# Car Classification Using CNN and Transfer Learning

### By: Fulgent Kvasir E. Lavesores


## Definition of the Problem

The goal of this project is to build a Convolutional Neural Network (CNN) that classifies car images into their respective makes and models. Using a pretrained model (ResNet50), I aim to fine-tune its layers to adapt it to a dataset containing car images with bounding boxes and class labels. The project seeks to:
1. Process the car dataset and prepare it for training.
2. Train the CNN model using a combination of transfer learning and custom classification layers.
3. Evaluate the model's performance on unseen test data.


## Data Acquisition

The dataset used in this project is the **Stanford Cars Dataset**, which includes:
- **Training Images**: 8,144 car images, each annotated with bounding boxes and class labels.
- **Testing Images**: 8,041 car images with similar annotations.

The dataset was loaded using the `scipy.io` library from `.mat` files. Bounding boxes were used to crop images before feeding them into the model.

### Source:
Stanford Cars Dataset ([https://www.kaggle.com/datasets/jessicali9530/stanford-cars-dataset/data](https://www.kaggle.com/datasets/jessicali9530/stanford-cars-dataset/data))


## Exploration and Analysis of Data

### Key Points:
1. **Number of Classes**: 196 car classes.
2. **Number of Images**:
   - Training: 8,144 images
   - Testing: 8,041 images
3. **Bounding Boxes**: Each image has a bounding box annotation for cropping the car region.
4. **Dataset Distribution**: Class distribution is balanced, with each class having a similar number of images.

### Descriptive Statistics:
Statistical analysis was performed to understand the data distribution. Key insights include:
- The bounding box coordinates vary across the dataset.
- The dataset covers a wide range of car models, providing good diversity.

### Visualizations:
Plots were created to visualize:
- Training loss and accuracy over epochs.
- Validation performance trends during training.


## Data Preparation

### Steps:
1. **Loading and Parsing Annotations**:
   - Annotations were loaded from `.mat` files for training and testing datasets.
   - Bounding box coordinates were extracted to crop images.
2. **Data Splitting**:
   - The training dataset was split into:
     - **Training Set**: 80% of the data
     - **Validation Set**: 20% of the data
3. **Preprocessing**:
   - Images were resized to 224x224 pixels (matching ResNet50 input size).
   - Pixel values were normalized to the range [0, 1].
4. **Data Augmentation**:
   - Techniques such as rotation, zooming, and horizontal flipping were applied to the training data.
5. **Model Preparation**:
   - Custom fully connected layers were added for classification.


### Loading the Libraries

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Loading of Dataset

In [ ]:
import os
import numpy as np
import pandas as pd
import scipy.io as sio
from PIL import Image
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

# Ustawienia, aby ukryć ostrzeżenia TensorFlow
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Wczytanie danych treningowych
cars_annos_train = sio.loadmat('../input/standford-cars-dataset-meta/devkit/cars_train_annos.mat')
annotations_train = cars_annos_train['annotations']
annotations_train = np.transpose(annotations_train)

fnames_train = []
bboxes_train = []

for annotation in annotations_train:
    bbox_x1 = annotation[0][0][0][0]
    bbox_y1 = annotation[0][1][0][0]
    bbox_x2 = annotation[0][2][0][0]
    bbox_y2 = annotation[0][3][0][0]
    fname = annotation[0][5][0]
    car_class = annotation[0][4][0]
    bboxes_train.append((fname, bbox_x1, bbox_x2, bbox_y1, bbox_y2, int(car_class[0])))

train_meta = pd.DataFrame(bboxes_train, columns=['fnames', 'bbox_x1', 'bbox_x2', 'bbox_y1', 'bbox_y2', 'car_class'])

# Wczytanie danych testowych
cars_annos_test = sio.loadmat('../input/standford-cars-dataset-meta/cars_test_annos_withlabels (1).mat')
annotations_test = cars_annos_test['annotations']
annotations_test = np.transpose(annotations_test)

fnames_test = []
bboxes_test = []

for annotation in annotations_test:
    bbox_x1 = annotation[0][0][0][0]
    bbox_y1 = annotation[0][1][0][0]
    bbox_x2 = annotation[0][2][0][0]
    bbox_y2 = annotation[0][3][0][0]
    fname = annotation[0][5][0]
    car_class = annotation[0][4][0]
    bboxes_test.append((fname, bbox_x1, bbox_x2, bbox_y1, bbox_y2, int(car_class[0])))

test_meta = pd.DataFrame(bboxes_test, columns=['fnames', 'bbox_x1', 'bbox_x2', 'bbox_y1', 'bbox_y2', 'car_class'])



In [ ]:
# Podział danych treningowych na zbiór treningowy i walidacyjny
train_meta, val_meta = np.split(train_meta.sample(frac=1, random_state=42), [int(0.8 * len(train_meta))])

# Tworzenie katalogów dla przetworzonych obrazów
os.makedirs('/kaggle/working/training', exist_ok=True)
os.makedirs('/kaggle/working/validation', exist_ok=True)
os.makedirs('/kaggle/working/testing', exist_ok=True)


### Cleaning the Data

In [6]:
# Przetwarzanie i zapisywanie obrazów treningowych
for i in range(len(train_meta)):
    img_path = f"../input/stanford-cars-dataset/cars_train/cars_train/{train_meta['fnames'].iloc[i]}"
    left, top, right, bottom = train_meta['bbox_x1'].iloc[i], train_meta['bbox_y1'].iloc[i], train_meta['bbox_x2'].iloc[i], train_meta['bbox_y2'].iloc[i]
    if right <= left or bottom <= top:
        print(f"Invalid bbox for image {img_path}: left={left}, top={top}, right={right}, bottom={bottom}")
        continue
    img = Image.open(img_path).crop((left, top, right, bottom))
    img_output = f"/kaggle/working/training/{train_meta['fnames'].iloc[i]}"
    img.save(img_output)

# Przetwarzanie i zapisywanie obrazów walidacyjnych
for i in range(len(val_meta)):
    img_path = f"../input/stanford-cars-dataset/cars_train/cars_train/{val_meta['fnames'].iloc[i]}"
    left, top, right, bottom = val_meta['bbox_x1'].iloc[i], val_meta['bbox_y1'].iloc[i], val_meta['bbox_x2'].iloc[i], val_meta['bbox_y2'].iloc[i]
    if right <= left or bottom <= top:
        print(f"Invalid bbox for image {img_path}: left={left}, top={top}, right={right}, bottom={bottom}")
        continue
    img = Image.open(img_path).crop((left, top, right, bottom))
    img_output = f"/kaggle/working/validation/{val_meta['fnames'].iloc[i]}"
    img.save(img_output)

# Przetwarzanie i zapisywanie obrazów testowych
for i in range(len(test_meta)):
    img_path = f"../input/stanford-cars-dataset/cars_test/cars_test/{test_meta['fnames'].iloc[i]}"
    left, top, right, bottom = test_meta['bbox_x1'].iloc[i], test_meta['bbox_y1'].iloc[i], test_meta['bbox_x2'].iloc[i], test_meta['bbox_y2'].iloc[i]
    if right <= left or bottom <= top:
        print(f"Invalid bbox for image {img_path}: left={left}, top={top}, right={right}, bottom={bottom}")
        continue
    img = Image.open(img_path).crop((left, top, right, bottom))
    img_output = f"/kaggle/working/testing/{test_meta['fnames'].iloc[i]}"
    img.save(img_output)

In [ ]:
# Aktualizacja ścieżek w DataFrame
train_meta['fnames'] = train_meta['fnames'].apply(lambda x: f"training/{x}")
val_meta['fnames'] = val_meta['fnames'].apply(lambda x: f"validation/{x}")
test_meta['fnames'] = test_meta['fnames'].apply(lambda x: f"testing/{x}")

# Konwersja kolumny 'car_class' na string
train_meta['car_class'] = train_meta['car_class'].astype(str)
val_meta['car_class'] = val_meta['car_class'].astype(str)
test_meta['car_class'] = test_meta['car_class'].astype(str)

### Selecting / Transforming / Standardizing Features

In [7]:
# Data augmentation dla zbioru treningowego
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest'
)

training_set = train_datagen.flow_from_dataframe(
    dataframe=train_meta,
    directory="/kaggle/working",
    x_col='fnames',
    y_col='car_class',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)
# Data augmentation dla zbioru walidacyjnego
val_datagen = ImageDataGenerator(rescale=1./255)

validation_set = val_datagen.flow_from_dataframe(
    dataframe=val_meta,
    directory="/kaggle/working",
    x_col='fnames',
    y_col='car_class',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

In [ ]:
# Normalizacja danych testowych
test_datagen = ImageDataGenerator(rescale=1./255)

test_set = test_datagen.flow_from_dataframe(
    dataframe=test_meta,
    directory="/kaggle/working",
    x_col='fnames',
    y_col='car_class',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)


In [ ]:
# Budowa modelu z użyciem ResNet50
from tensorflow.keras.applications import ResNet50
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Odmrażanie ostatnich kilku warstw modelu bazowego
for layer in base_model.layers[-10:]:
    layer.trainable = True

model = Sequential([
    base_model,
    Flatten(),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(196, activation='softmax')  # 196 klas
])

### Training Model

In [8]:
# Kompilacja modelu z mniejszym współczynnikiem uczenia
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Trening modelu
history = model.fit(training_set, validation_data=validation_set, epochs=30, callbacks=[early_stopping])


In [9]:

# Wizualizacja wyników
plt.figure(figsize=(14, 5))

# Wykres strat
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

# Wykres dokładności
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import seaborn as sns

def evaluate_model(y_true, y_pred):
    metrics = {
        'Accuracy': accuracy_score(y_true, y_pred),
        'Precision': precision_score(y_true, y_pred, average='macro'),
        'Recall': recall_score(y_true, y_pred, average='macro'),
        'F1-Score': f1_score(y_true, y_pred, average='macro')
    }
    return metrics

def plot_metrics(metrics, title='Model Performance'):
    fig, ax = plt.subplots(figsize=(8, 5))
    metrics_names = list(metrics.keys())
    values = list(metrics.values())
    ax.bar(metrics_names, values, color='skyblue')
    ax.set_xlabel('Metrics')
    ax.set_ylabel('Scores')
    ax.set_ylim(0, 1.1)
    ax.set_title(title)
    plt.show()

def test_model(model, validation_set, test_set):
    # Predictions using the dataset objects
    y_pred_val = np.argmax(model.predict(validation_set), axis=-1)
    y_pred_test = np.argmax(model.predict(test_set), axis=-1)

    # Extracting true labels from your dataset
    y_true_val = np.concatenate([y for x, y in validation_set], axis=0)
    y_true_test = np.concatenate([y for x, y in test_set], axis=0)

    # Evaluate
    val_metrics = evaluate_model(y_true_val, y_pred_val)
    test_metrics = evaluate_model(y_true_test, y_pred_test)

    # Plot results
    plot_metrics(val_metrics, title='Validation Set Performance')
    plot_metrics(test_metrics, title='Test Set Performance')

# Replace 'history' with 'model', assuming 'model' is your trained model object
test_model(model, validation_set, test_set)


### Data Visualizations